In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/car-price-prediction-ru/sample_submission.csv
/kaggle/input/car-price-prediction-ru/train.csv
/kaggle/input/car-price-prediction-ru/test.csv


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [4]:
train_data = pd.read_csv("/kaggle/input/car-price-prediction-ru/train.csv")
train_data.head(10)

,car_id,car_mark,year,mileage,engine,drive,body,color,transmission,brand,price
0,63407,Subaru Legacy I,1994,250 000 км,2.2 л / 136 л.с. / Бензин,полный,седан,серебристый,автомат,Subaru,85000
1,15174,Daewoo Nexia I,1998,262 000 км,1.5 л / 75 л.с. / Бензин,передний,седан,красный,механика,Daewoo,90000
2,40267,Mazda CX-5 I Рестайлинг,2016,95 000 км,2.0 л / 150 л.с. / Бензин,полный,внедорожник 5 дв.,белый,автомат,Mazda,1695000
3,31131,Kia Seltos I,2022,Новый,2.0 л / 149 л.с. / Бензин,передний,внедорожник 5 дв.,Prestige,вариатор,Kia,2284900
4,55806,Renault Logan II Рестайлинг,2022,Новый,1.6 л / 82 л.с. / Бензин,передний,седан,Life,механика,Renault,921300
5,40215,Mazda 6 I (GG) Рестайлинг,2006,189 600 км,2.0 л / 147 л.с. / Бензин,передний,лифтбек,серебристый,автомат,Mazda,369000
6,19362,Ford S-MAX I,2008,200 000 км,2.3 л / 161 л.с. / Бензин,передний,минивэн,серый,автомат,Ford,700000
7,50302,Nissan Primera III (P12),2004,300 000 км,1.8 л / 116 л.с. / Бензин,передний,седан,синий,автомат,Nissan,300000
8,25455,Hyundai Solaris II,2019,115 094 км,1.6 л / 123 л.с. / Бензин,передний,седан,белый,автомат,Hyundai,978000
9,7374,BMW X5 30d II (E70),2008,272 000 км,3.0 л / 235 л.с. / Дизель,полный,внедорожник 5 дв.,серый,автомат,BMW,1560000


In [5]:
train_data.isnull().sum()

car_id          0
car_mark        0
year            0
mileage         0
engine          0
drive           0
body            0
color           0
transmission    0
brand           0
price           0
dtype: int64

In [6]:
train_data.columns

Index(['car_id', 'car_mark', 'year', 'mileage', 'engine', 'drive', 'body',
       'color', 'transmission', 'brand', 'price'],
      dtype='object')

In [7]:
train_data = train_data[['mileage', 'car_id', 'brand', 'transmission', 'price']]

In [8]:
train_data.mileage

0        250 000 км
1        262 000 км
2         95 000 км
3             Новый
4             Новый
            ...    
57061     47 224 км
57062         Новый
57063         Новый
57064    170 000 км
57065    164 000 км
Name: mileage, Length: 57066, dtype: object

In [9]:
# Assuming your train data is stored in a pandas DataFrame called 'train_data'
# and the column containing the mileage data is named 'mileage'
mileage_series = train_data['mileage']
mileage_series = mileage_series.astype(str).str.replace('[^0-9]', '', regex=True)
mileage_series = pd.to_numeric(mileage_series, errors='coerce')

print(mileage_series)

0        250000.0
1        262000.0
2         95000.0
3             NaN
4             NaN
           ...   
57061     47224.0
57062         NaN
57063         NaN
57064    170000.0
57065    164000.0
Name: mileage, Length: 57066, dtype: float64


In [10]:
# Calculate the average of non-NaN values
average_value = mileage_series.mean()

# Fill NaN values with the average value
mileage_series = mileage_series.fillna(average_value)
mileage_series = mileage_series.astype('int64')
print(mileage_series)


0        250000
1        262000
2         95000
3        127346
4        127346
          ...  
57061     47224
57062    127346
57063    127346
57064    170000
57065    164000
Name: mileage, Length: 57066, dtype: int64


In [11]:
train_data["mileage"] = mileage_series

In [12]:
train_data

,mileage,car_id,brand,transmission,price
0,250000,63407,Subaru,автомат,85000
1,262000,15174,Daewoo,механика,90000
2,95000,40267,Mazda,автомат,1695000
3,127346,31131,Kia,вариатор,2284900
4,127346,55806,Renault,механика,921300
...,...,...,...,...,...
57061,47224,25365,Hyundai,автомат,1079000
57062,127346,48056,Mitsubishi,вариатор,2818000
57063,127346,59011,Skoda,механика,1253050
57064,170000,67224,Toyota,механика,600000


In [13]:
train_data.car_id

0        63407
1        15174
2        40267
3        31131
4        55806
         ...  
57061    25365
57062    48056
57063    59011
57064    67224
57065    71530
Name: car_id, Length: 57066, dtype: int64

In [14]:
# Assuming your 'car_id' column is stored in a pandas Series called 'car_id_series'
car_id_series = train_data['car_id']

# Convert 'car_id' to a sequential basis
sequential_car_id = car_id_series.rank(method='dense')

print(sequential_car_id)

0        47541.0
1        11390.0
2        30241.0
3        23434.0
4        41895.0
          ...   
57061    19092.0
57062    36075.0
57063    44273.0
57064    50367.0
57065    53610.0
Name: car_id, Length: 57066, dtype: float64


In [15]:
train_data["car_id"] = sequential_car_id 

In [16]:
label_encoder = LabelEncoder()
train_data['brand'] = label_encoder.fit_transform(train_data['brand'])

In [17]:
label_encoder = LabelEncoder()
train_data['transmission'] = label_encoder.fit_transform(train_data['transmission'])

In [18]:
train_data

,mileage,car_id,brand,transmission,price
0,250000,47541.0,104,62,85000
1,262000,11390.0,23,64,90000
2,95000,30241.0,75,62,1695000
3,127346,23434.0,61,63,2284900
4,127346,41895.0,93,64,921300
...,...,...,...,...,...
57061,47224,19092.0,53,62,1079000
57062,127346,36075.0,79,63,2818000
57063,127346,44273.0,101,64,1253050
57064,170000,50367.0,108,64,600000


In [19]:
test_data = pd.read_csv("/kaggle/input/car-price-prediction-ru/test.csv")
test_data.head()

,car_id,car_mark,year,mileage,engine,drive,body,color,transmission,brand
0,71542,Volkswagen Tiguan II,2017,80 277 км,2.0 л / 150 л.с. / Дизель,полный,внедорожник 5 дв.,белый,робот,Volkswagen
1,29263,Infiniti G G25 IV,2010,224 000 км,2.5 л / 222 л.с. / Бензин,задний,седан,серый,автомат,Infiniti
2,27118,Hyundai Creta II,2022,Новый,2.0 л / 150 л.с. / Бензин,передний,внедорожник 5 дв.,Family,автомат,Hyundai
3,24436,Honda Civic VII,2001,194 000 км,1.7 л / 125 л.с. / Бензин,передний,седан,зелёный,автомат,Honda
4,28254,Hyundai Creta I Рестайлинг,2021,15 500 км,1.6 л / 121 л.с. / Бензин,полный,внедорожник 5 дв.,серебристый,механика,Hyundai


In [20]:
test_data = test_data[['car_id', 'mileage', 'transmission', 'brand']]

In [21]:
train_data.isnull().sum()

mileage         0
car_id          0
brand           0
transmission    0
price           0
dtype: int64

In [22]:
# Assuming your test data is stored in a pandas DataFrame called 'test_data'
# and the column containing the mileage data is named 'mileage'
# I named it milage_seri 
mileage_seri = test_data['mileage']
mileage_seri = mileage_seri.astype(str).str.replace('[^0-9]', '', regex=True)
mileage_seri = pd.to_numeric(mileage_seri, errors='coerce')

print(mileage_seri)

0         80277.0
1        224000.0
2             NaN
3        194000.0
4         15500.0
           ...   
19018    139000.0
19019    105000.0
19020    218000.0
19021        16.0
19022    136726.0
Name: mileage, Length: 19023, dtype: float64


In [23]:
# Calculate the average of non-NaN values
average_value = mileage_seri.mean()

# Fill NaN values with the average value
mileage_seri = mileage_seri.fillna(average_value)
mileage_seri = mileage_seri.astype('int64')
print(mileage_seri)


0         80277
1        224000
2        127746
3        194000
4         15500
          ...  
19018    139000
19019    105000
19020    218000
19021        16
19022    136726
Name: mileage, Length: 19023, dtype: int64


In [24]:
test_data["mileage"] = mileage_seri

In [25]:
label_encoder = LabelEncoder()
test_data['transmission'] = label_encoder.fit_transform(test_data['transmission'])

In [26]:
label_encoder = LabelEncoder()
test_data['brand'] = label_encoder.fit_transform(test_data['brand'])

In [27]:
test_data

,car_id,mileage,transmission,brand
0,71542,80277,50,99
1,29263,224000,47,46
2,27118,127746,47,45
3,24436,194000,47,42
4,28254,15500,49,45
...,...,...,...,...
19018,70809,139000,47,99
19019,51222,105000,49,74
19020,38951,218000,47,58
19021,66068,16,47,97


In [28]:
X_train = train_data.drop(columns=["price"])
y_train = train_data["price"]

In [29]:
n_estimators = 100
max_depth = 5
min_samples_split = 10


In [30]:
rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split)

In [31]:
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, min_samples_split=10)

In [32]:
# Reorder the test_data columns to match the order of X_train columns
test_data = test_data[X_train.columns]

In [33]:
predictions = rf.predict(test_data)

In [34]:
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [35]:
n_estimators = 100
max_depth = 5
min_samples_split = 10

In [36]:
rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split)
rf.fit(X_train_split, y_train_split)

RandomForestClassifier(max_depth=5, min_samples_split=10)

In [37]:
val_predictions = rf.predict(X_val_split)

In [38]:
# Calculate the mean squared error (MSE)
mse = mean_squared_error(y_val_split, val_predictions)

# Take the natural logarithm of the MSE
ln_mse = np.log(mse)

# Calculate the mean of the squared logarithmic errors
mean_squared_ln_error = np.mean(ln_mse)

# Take the square root of the mean squared logarithmic error to get RLMSE
rlmse = np.sqrt(mean_squared_ln_error)

print("RLMSE:", rlmse)

RLMSE: 5.51266486003536


In [39]:
def calculate_rlmse(true_values, predicted_values):
    # Calculate the mean squared error (MSE)
    mse = np.mean((y_val_split, val_predictions) ** 2)
    
    # Take the natural logarithm of the MSE
    ln_mse = np.log(mse)
    
    # Calculate the mean of the squared logarithmic errors
    mean_squared_ln_error = np.mean(ln_mse)
    
    # Take the square root of the mean squared logarithmic error to get RLMSE
    rlmse = np.sqrt(mean_squared_ln_error)
    
    return rlmse

In [40]:
accuracy = accuracy_score(y_val_split, val_predictions)
print("Validation Accuracy:", accuracy)

Validation Accuracy: 0.040301384264937797


In [41]:
submission = pd.DataFrame({"car_id": test_data["car_id"], "predicted_price": predictions})
submission.to_csv("submission.csv", index=False)